In [1]:
%load_ext autoreload
%matplotlib inline
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive

from regressor_analysis import *

In [3]:
calcium_video_fname   = "/Users/jordan/Downloads/Regressor Analysis/May.1.19_huc6f_5dpf_PAG_lv30_1A_z_0_mc.tif"
roi_data_fname        = "/Users/jordan/Downloads/Regressor Analysis/May.1.19_huc6f_5dpf_PAG_lv30_1A_z_0_mc/roi_data.npy"
bout_fname            = "/Users/jordan/Downloads/Regressor Analysis/May.1.19_1A_behaviors.csv"
frame_timestamp_fname = "/Users/jordan/Downloads/Regressor Analysis/2019.9.17 1A - timestamps.txt"

In [4]:
calcium_fps         = 3
tail_fps            = 349.0
tail_calcium_offset = 0

In [5]:
(correlation_results, regression_coefficients,
 regression_intercepts, regression_scores, regressors,
 spatial_footprints, temporal_footprints,
 calcium_video, mean_images, n_frames, roi_centers) = regressor_analysis(calcium_video_fname, roi_data_fname, bout_fname,
                                                                         frame_timestamp_fname, tail_calcium_offset)

In [6]:
def show_correlation_plot(z, max_p, regressor):
    plot_correlation(correlation_results, regressors, spatial_footprints, temporal_footprints, calcium_video, mean_images, n_frames, roi_centers, z, max_p, regressor, fig=None)
    plt.show()
    
interactive_plot = interactive(show_correlation_plot, z=(0, calcium_video.shape[1]-1), max_p=(0.01, 0.5), regressor=(0, len(regressors)-1))
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(IntSlider(value=0, description='z', max=0), FloatSlider(value=0.255, description='max_p'…

In [7]:
display(widgets.Label("Select which regressors to use for multilinear regression:"))
keys = list(regressors.keys())
checkboxes = []
for i in range(len(keys)):
    w = widgets.Checkbox(
        value=True,
        description=keys[i],
        disabled=False
    )
#     display(w)
    checkboxes.append(w)
box = widgets.HBox(children=checkboxes, width='200px', layout=widgets.Layout(flex_flow='row wrap'))
display(box)

Label(value='Select which regressors to use for multilinear regression:')

In [8]:
def show_multilinear_regression_plot(z):
    # check which regressors are selected
    selected_keys = [ keys[i] for i in range(len(keys)) if checkboxes[i].value ]
    selected_regressors = {k:regressors[k] for k in selected_keys if k in regressors}
    
    regression_coefficients, regression_intercepts, regression_scores = multilinear_regression(selected_regressors, temporal_footprints)
    
    plot_multilinear_regression(regression_coefficients, regression_intercepts, regression_scores, selected_regressors, spatial_footprints, temporal_footprints, calcium_video, mean_images, n_frames, roi_centers, z, fig=None)
    plt.show()
    
interactive_plot = interactive(show_multilinear_regression_plot, z=(0, calcium_video.shape[1]-1))
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(IntSlider(value=0, description='z', max=0), Output()), _dom_classes=('widget-interact',)…